In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
from functools import reduce

from IPython.display import display

# from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Drop rows or columns with >20% missingness
def clean_df(df):
    for x in df.columns:
        if (df[x].isnull().sum(axis=0)) > len(df) * 0.2:
            df.drop(columns=[x], inplace=True)
    for x in df.index:
        if df.loc[x, :].isnull().sum() > df.shape[1] * 0.2:
            df.drop(index=x, inplace=True)
    return df

In [ ]:
## Client Info


In [ ]:
clients = pd.read_csv("../data/arivale_snapshot_ISB_2019-05-10_0053/clients.tsv", skiprows=13, sep='\t', converters={'public_client_id': str})
print(clients.shape)
clients.head()

In [ ]:
clients = clients[['public_client_id','sex','race','age']]

In [ ]:
clients_clean = clean_df(clients)
print(clients_clean.shape)
clients_clean.head()

In [ ]:
## Frailty Measures

In [ ]:
fr_measures = pd.read_csv("../data/frailty/combination_fi_040124.csv")
print(fr_measures.shape)
fr_measures.head()

In [ ]:
fr_sub = fr_measures[['public_client_id','merge_fi']]

In [ ]:
fr_clean = clean_df(fr_sub)
print (fr_clean.shape)
fr_clean.head()

In [ ]:
## Metabolomics

In [ ]:
# mets = pd.read_csv("Metabolite_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str, 'MElightcyan': float, 'MEgreenyellow': float, 'MEpurple': float, 'MEblue': float, 'MEbrown': float, 'MEgreen': float,'MEmidnightblue': float, 'MEcyan': float, 'MEtan': float, 'MEyellow': float, 'MEblack': float, 'MEmagenta': float, 'MEred': float, 'MEsalmon': float, 'MEpink': float, 'MEturquoise': float, 'MEgrey': float})
mets = pd.read_csv("../Useful_Files/Metabolite_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str})
print(mets.shape)
mets.head()

In [ ]:
col_names = ['MElightcyan', 'MEgreenyellow', 'MEpurple', 'MEblue', 'MEbrown', 'MEgreen','MEmidnightblue','MEcyan','MEtan','MEyellow','MEblack','MEmagenta','MEred','MEsalmon','MEpink','MEturquoise','MEgrey']

mets[col_names] = mets[col_names].apply(pd.to_numeric, errors='coerce')

In [ ]:
print(mets.shape)

In [ ]:
missing_values = mets.isnull().sum()

# Calculate total number of entries in the DataFrame
total_entries = mets.shape[0] * mets.shape[1]

# Calculate percentage of missing values for each column
percentage_missing_values = (missing_values / total_entries) * 100

print("Percentage of missing values in each column:")
print(percentage_missing_values)

In [ ]:
mets_clean = clean_df(mets)
print(mets_clean.shape)
mets_clean.head()

In [ ]:
## Proteomics

In [ ]:
prots = pd.read_csv("../Useful_Files/Protein_module_eigenvalues.tsv", sep='\t', converters={'public_client_id': str, 'MEturquoise': float, 'MEblue': float, 'MEbrown': float, 'MEgrey': float})
print(prots.shape)
prots.head()

In [ ]:
col_names = ['MEturquoise', 'MEblue', 'MEbrown', 'MEgrey']
new_names = {col: col + '_prots' for col in col_names}
prots = prots.rename(columns=new_names)
print(prots.shape)
prots.head()

In [ ]:
## Labs

In [ ]:
labs = pd.read_csv("../data/arivale_snapshot_ISB_2019-05-10_0053/chemistries.tsv", skiprows=13, sep='\t', converters={'public_client_id': str})
print(labs.shape)
labs.head()

In [ ]:
# Get lab data at baseline
labs_baseline = labs[labs.days_since_first_draw <= 60]
labs_baseline.head()

In [ ]:
labs_fast_values = labs_baseline['fasting'].value_counts()
print(labs_fast_values)

In [ ]:
labs_fast = labs_baseline[labs_baseline['fasting'] == True]

In [ ]:
labs_fast_values = labs_fast['fasting'].value_counts()
print(labs_fast_values)

In [ ]:
labs_drop = labs_fast[labs_fast.vendor == 'LCA'].drop(['vendor_observation_id','fasting','observation_id','days_since_first_call','days_in_program',
                                                                     'days_since_first_draw','month','weekday', 'season', 'vendor','reflexive'],
                                                                   axis=1)
print(labs_drop.shape)
labs_drop.head()

In [ ]:
labs_clean = clean_df(labs_drop)
print(labs_clean.shape)
labs_clean.head()

In [ ]:
dfs_to_merge = [clients_clean, fr_clean, mets_clean, prots, labs_clean]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='public_client_id'), dfs_to_merge)
merged_df = merged_df.drop_duplicates(subset=['public_client_id'])

print(merged_df.shape)
merged_df.head()

In [ ]:
cat_cols = merged_df.select_dtypes(include='object').columns.tolist()
print(cat_cols)

In [ ]:
sex_cats = merged_df['sex'].value_counts()
print(sex_cats)

In [ ]:
merged_df['sex'].replace('F', 0,inplace=True)
merged_df['sex'].replace('M', 1,inplace=True)

In [ ]:
race_cats = merged_df['race'].value_counts()
print(race_cats)

In [ ]:
low_count_categories = race_cats[race_cats <= 20].index.tolist()
print(low_count_categories)

In [ ]:
merged_df.loc[merged_df['race'].isin(low_count_categories), 'race'] = 'other'
race_cats = merged_df['race'].value_counts()
print(race_cats)

In [ ]:
merged_df = pd.get_dummies(merged_df, columns=['race'])

In [ ]:
print(merged_df.shape)
merged_df.head()

In [ ]:
clients_removed = merged_df.merge(fd, how='left', on='public_client_id')
print(clients_removed.shape)

In [ ]:
merged_clean.to_csv('AR_cleaned_ISB_aging_new_merge_fi_wgcna.csv', index=False)

In [ ]:
df = pd.read_csv("AR_cleaned_ISB_aging_new_merge_fi_wgcna.csv")
print(df.shape)